<a href="https://colab.research.google.com/github/muhamad-hammad/chrurnRatePredictor/blob/main/CustomerChurnPrediction(ANN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
rjmanoj_credit_card_customer_churn_prediction_path = kagglehub.dataset_download('rjmanoj/credit-card-customer-churn-prediction')

print('Data source import complete.')


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv


In [3]:
df = pd.read_csv('/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv')

In [4]:
df.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [5]:
df.shape

(10000, 14)

In [6]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [8]:
df.duplicated().sum()


np.int64(0)

In [9]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [10]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [11]:
df['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [12]:
df = df.drop(columns = ['RowNumber', 'CustomerId', 'Surname'])

In [13]:
df.head()

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42       2       0.00              1   
1          608     Spain  Female   41       1   83807.86              1   
2          502    France  Female   42       8  159660.80              3   
3          699    France  Female   39       1       0.00              2   
4          850     Spain  Female   43       2  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0          1               1        101348.88       1  
1          0               1        112542.58       0  
2          1               0        113931.57       1  
3          0               0         93826.63       0  
4          1               1         79084.10       0

In [14]:
df = pd.get_dummies(df,columns=['Geography', 'Gender'], drop_first=True)

In [15]:
df.head()

CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619   42       2       0.00              1          1   
1          608   41       1   83807.86              1          0   
2          502   42       8  159660.80              3          1   
3          699   39       1       0.00              2          0   
4          850   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  Geography_Germany  \
0               1        101348.88       1              False   
1               1        112542.58       0              False   
2               0        113931.57       1              False   
3               0         93826.63       0              False   
4               1         79084.10       0              False   

   Geography_Spain  Gender_Male  
0            False        False  
1             True        False  
2            False        False  
3            False        False  
4             True        False

In [16]:
from sklearn.model_selection import train_test_split
X = df.drop(columns='Exited')
y = df['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
X


CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619   42       2       0.00              1          1   
1             608   41       1   83807.86              1          0   
2             502   42       8  159660.80              3          1   
3             699   39       1       0.00              2          0   
4             850   43       2  125510.82              1          1   
...           ...  ...     ...        ...            ...        ...   
9995          771   39       5       0.00              2          1   
9996          516   35      10   57369.61              1          1   
9997          709   36       7       0.00              1          0   
9998          772   42       3   75075.31              2          1   
9999          792   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Geography_Germany  Geography_Spain  \
0                  1        101348.88              False            False   
1                  1        112542.58              False             True   
2                  0        113931.57              False            False   
3                  0         93826.63              False            False   
4                  1         79084.10              False             True   
...              ...              ...                ...              ...   
9995               0         96270.64              False            False   
9996               1        101699.77              False            False   
9997               1         42085.58              False            False   
9998               0         92888.52               True            False   
9999               0         38190.78              False            False   

      Gender_Male  
0           False  
1           False  
2           False  
3           False  
4           False  
...           ...  
9995         True  
9996         True  
9997        False  
9998         True  
9999        False  

[10000 rows x 11 columns]

In [18]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [19]:
X_train.shape

(8000, 11)

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()
X_trainScaled = scaler.fit_transform(X_train)
X_testScaled = scaler.transform(X_test)

In [22]:
X_trainScaled

array([[ 0.35649971, -0.6557859 ,  0.34567966, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [-0.20389777,  0.29493847, -0.3483691 , ...,  1.72572313,
        -0.57638802,  0.91324755],
       [-0.96147213, -1.41636539, -0.69539349, ..., -0.57946723,
         1.73494238,  0.91324755],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ..., -0.57946723,
        -0.57638802, -1.09499335],
       [ 0.15932282,  0.3900109 ,  1.03972843, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [ 0.47065475,  1.15059039, -1.38944225, ...,  1.72572313,
        -0.57638802,  0.91324755]])

In [23]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [24]:
model = Sequential()

model.add(Dense(11, activation='sigmoid', input_dim=11))
model.add(Dense(11, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276 (1.08 KB)

 Trainable params: 276 (1.08 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [27]:
history = model.fit(X_trainScaled, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8021 - loss: 0.5072 - val_accuracy: 0.7987 - val_loss: 0.4666
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7903 - loss: 0.4687 - val_accuracy: 0.8025 - val_loss: 0.4331
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7993 - loss: 0.4438 - val_accuracy: 0.8181 - val_loss: 0.4226
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8049 - loss: 0.4309 - val_accuracy: 0.8225 - val_loss: 0.4192
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8073 - loss: 0.4369 - val_accuracy: 0.8225 - val_loss: 0.4173
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8027 - loss: 0.4380 - val_accuracy: 0.8206 - val_loss: 0.4166
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8038 - loss: 0.4418 - val_accuracy: 0.8225 - val_loss: 0.4154
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8068 - loss: 0.4292 - val_accu

In [28]:
model.layers[0].get_weights()

[array([[ 3.35281268e-02, -1.67526320e-01,  5.96762076e-02,
         -5.84980883e-02, -5.22919118e-01, -5.76228797e-01,
          1.15673184e-01, -9.29745436e-02,  2.86777616e-01,
          4.94386464e-01, -2.01567873e-01],
        [ 1.45928517e-01,  8.73361290e-01, -4.90022779e-01,
         -1.68855035e+00, -2.09693313e+00,  2.12104857e-01,
          3.70065570e-01,  7.87206471e-01,  3.56751174e-01,
          2.23942399e-01,  1.46894884e+00],
        [-5.16779646e-02,  3.99323016e-01, -5.46031557e-02,
          1.07384332e-01, -1.84782445e-01, -5.57932407e-02,
          6.04736768e-02,  1.10760637e-01, -8.07145461e-02,
          3.28021675e-01, -1.68591917e-01],
        [ 9.97178629e-02, -4.85560209e-01,  6.19239211e-01,
          5.22471547e-01,  4.18907046e-01, -4.57423590e-02,
          3.30430299e-01,  3.85731280e-01, -4.89423394e-01,
         -8.20949912e-01,  8.99651870e-02],
        [-3.24167728e-01,  5.04925132e-01,  1.09974480e+00,
          3.00111294e-01, -1.86709613e-01, -

In [29]:
y_log = model.predict(X_testScaled)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [30]:
y_pred = np.where(y_log>0.5,1,0)

In [31]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_test))

0.862


In [32]:
import matplotlib.pyplot as plt

In [33]:
history.history

{'accuracy': [0.7934374809265137,
  0.7934374809265137,
  0.8042187690734863,
  0.8064062595367432,
  0.8064062595367432,
  0.8056250214576721,
  0.8089062571525574,
  0.8082812428474426,
  0.8149999976158142,
  0.8151562213897705,
  0.8178125023841858,
  0.8192187547683716,
  0.8196874856948853,
  0.8245312571525574,
  0.8259375095367432,
  0.828906238079071,
  0.83203125,
  0.836718738079071,
  0.8356249928474426,
  0.8392187356948853,
  0.8415625095367432,
  0.8434374928474426,
  0.8442187309265137,
  0.8470312356948853,
  0.8460937738418579,
  0.8515625,
  0.8518750071525574,
  0.8520312309265137,
  0.8539062738418579,
  0.8534374833106995,
  0.8567187786102295,
  0.8575000166893005,
  0.8576562404632568,
  0.8581249713897705,
  0.8592187762260437,
  0.8568750023841858,
  0.8603125214576721,
  0.8590624928474426,
  0.8589062690734863,
  0.8592187762260437,
  0.8603125214576721,
  0.8587499856948853,
  0.8614062666893005,
  0.8614062666893005,
  0.8614062666893005,
  0.8600000143051

In [34]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [35]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])